In [1]:
# See: https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/5_DataManagement/tensorflow_dataset_api.py
from __future__ import print_function
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import glob

In [2]:
# Data set

# Our music genre labels
label_dict = {
    'Classical': 0,
    'Electronic': 1,
    'Pop': 2,
    'HipHop': 3,
    'Metal': 4,
    'Rock': 5
}

NUM_CLASSES = len(label_dict)

def input_parser(img_path, label_value):
    label = tf.one_hot(label_value, NUM_CLASSES)
    img_file = tf.read_file(img_path)
    img_decoded = tf.cast(tf.image.decode_image(img_file, channels=1), tf.float32)

    return img_decoded, label

def load_data():
    data = []
    for file in glob.glob("data/*/**.png"):
        # extract label
        filename = file.split("/")[len(file.split("/")) - 1]
        genre = filename.split("_")[0]

        # if we can't extract the label from the image we should not train on it
        if genre not in label_dict:
            continue;
        
        label_val = label_dict.get(genre)
        data.append((file, label_val))
    
    #random.shuffle(data)
    return data

data = load_data()
images = [x[0] for x in data]
labels = [x[1] for x in data]

# create TensorFlow Dataset objects
tf_data = tf.contrib.data.Dataset.from_tensor_slices((images, labels))
tf_data = tf_data.map(input_parser)    

Instructions for updating:
Use `tf.data.Dataset.from_tensor_slices()`.


In [3]:
# Parameters
learning_rate = 0.001
num_steps = 2000
batch_size = 128
display_step = 100

# Network Parameters
n_input = 16384 # 128*128
n_classes = len(label_dict)
dropout = 0.75 # Dropout, probability to keep units

In [4]:
sess = tf.Session()

# Create batches of data
tf_data = tf_data.batch(batch_size)
# Create an iterator, to go over the dataset
#iterator = tf_data.make_initializable_iterator()
# It is better to use 2 placeholders, to avoid to load all data into memory,
# and avoid the 2Gb restriction length of a tensor.
#_data = tf.placeholder(tf.float32, [None, n_input])
#_labels = tf.placeholder(tf.float32, [None, n_classes])
# Initialize the iterator
#sess.run(iterator.initializer, feed_dict={_data: images,_labels: labels})
iterator = tf_data.make_one_shot_iterator()

# Neural Net Input
X, Y = iterator.get_next()

In [5]:
# -----------------------------------------------
# THIS IS A CLASSIC CNN (see examples, section 3)
# -----------------------------------------------
# Note that a few elements have changed (usage of sess run).

# Create model
def conv_net(x, n_classes, dropout, reuse, is_training):
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
        # Reshape to match picture format [Height x Width x Channel]
        # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
        #x = tf.reshape(x, shape=[-1, 128, 128, 1])

        # Convolution Layer with 32 filters and a kernel size of 5
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

        # Convolution Layer with 32 filters and a kernel size of 5
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)

        # Flatten the data to a 1-D vector for the fully connected layer
        fc1 = tf.contrib.layers.flatten(conv2)

        # Fully connected layer (in contrib folder for now)
        fc1 = tf.layers.dense(fc1, 1024)
        # Apply Dropout (if is_training is False, dropout is not applied)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

        # Output layer, class prediction
        out = tf.layers.dense(fc1, n_classes)
        # Because 'softmax_cross_entropy_with_logits' already apply softmax,
        # we only apply softmax to testing network
        out = tf.nn.softmax(out) if not is_training else out

    return out


In [6]:
# Because Dropout have different behavior at training and prediction time, we
# need to create 2 distinct computation graphs that share the same weights.

# Create a graph for training
logits_train = conv_net(X, n_classes, dropout, reuse=False, is_training=True)
# Create another graph for testing that reuse the same weights, but has
# different behavior for 'dropout' (not applied).
logits_test = conv_net(X, n_classes, dropout, reuse=True, is_training=False)

# Define loss and optimizer (with train logits, for dropout to take effect)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits_train, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits_test, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Run the initializer
sess.run(init)

ValueError: Input 0 of layer conv2d_1 is incompatible with the layer: its rank is undefined, but the layer requires a defined rank.

In [ ]:
# Training cycle
for step in range(1, num_steps + 1):

    try:
        # Run optimization
        sess.run(train_op)
    except tf.errors.OutOfRangeError:
        # Reload the iterator when it reaches the end of the dataset
        sess.run(iterator.initializer,
                 feed_dict={_data: mnist.train.images,
                            _labels: mnist.train.labels})
        sess.run(train_op)

    if step % display_step == 0 or step == 1:
        # Calculate batch loss and accuracy
        # (note that this consume a new batch of data)
        loss, acc = sess.run([loss_op, accuracy])
        print("Step " + str(step) + ", Minibatch Loss= " + \
              "{:.4f}".format(loss) + ", Training Accuracy= " + \
              "{:.3f}".format(acc))

print("Optimization Finished!")